In [1]:
## pip install google-cloud-bigquery pandas scikit-learn
## pip install db-dtypes


In [2]:
from google.cloud import bigquery
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import os

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key-path.json"

In [4]:
client = bigquery.Client(project="article1-data-pipeline")

### Modèle de recommandation mentorat

In [23]:
# === 1. Charger les profils des jeunes ===
query_jeunes = """
SELECT
  j_id,
  CONCAT(COALESCE(secteur_1, ''), ' ', COALESCE(besoins, ''), ' ', COALESCE(passions, '')) AS profil_jeune
FROM `article1-data-pipeline.article1_dbt_staging.stg_jeunes`
WHERE statut = 'EN_ATTENTE_ACTIVATION'
"""

# === 2. Charger les profils des bénévoles ===
query_benevoles = """
SELECT
  bv_id,
  CONCAT(COALESCE(poste_actuel, ''), ' ', COALESCE(type_aide, ''), ' ', COALESCE(experience, ''), ' ', COALESCE(passions, '')) AS profil_benevole
FROM `article1-data-pipeline.article1_dbt_staging.stg_benevoles`
WHERE statut = 'APTE'
"""

jeunes_df = client.query(query_jeunes).to_dataframe()
benevoles_df = client.query(query_benevoles).to_dataframe()

# === 3. Vectorisation texte ===
vectorizer = TfidfVectorizer()
tfidf_jeunes = vectorizer.fit_transform(jeunes_df['profil_jeune'])
tfidf_benevoles = vectorizer.transform(benevoles_df['profil_benevole'])

# === 4. Similarité cosinus ===
similarities = cosine_similarity(tfidf_jeunes, tfidf_benevoles)

# === 5. Top recommandations ===
results = []
for i, j_id in enumerate(jeunes_df['j_id']):
    top_benevole_index = similarities[i].argmax()
    top_score = similarities[i][top_benevole_index]
    results.append({
        'j_id': j_id,
        'bv_id_recommande': benevoles_df.iloc[top_benevole_index]['bv_id'],
        'score_similarite': round(top_score, 3)
    })

reco_df = pd.DataFrame(results)
print(reco_df.head())

C:\Users\awadi\anaconda3\lib\site-packages\google\cloud\bigquery\table.py:2551: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
C:\Users\awadi\anaconda3\lib\site-packages\google\cloud\bigquery\table.py:2565: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
C:\Users\awadi\anaconda3\lib\site-packages\google\cloud\bigquery\table.py:2579: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
C:\Users\awadi\anaconda3\lib\site-packages\google\cloud\bigquery\table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with t

     j_id bv_id_recommande  score_similarite
0  190833            69827             0.583
1  191021            69484             0.122
2  191226            70144             0.701
3  191939            69484             1.000
4   19518            69579             0.000


In [24]:
# Supprimer les scores trop faibles (facultatif)
reco_df = reco_df[reco_df['score_similarite'] > 0.2]
reco_df

,j_id,bv_id_recommande,score_similarite
0,190833,69827,0.583
2,191226,70144,0.701
3,191939,69484,1.000
6,195609,69484,1.000
7,195782,69484,1.000
...,...,...,...
4912,221071,69484,1.000
4913,223615,69484,1.000
4914,223618,69484,1.000
4915,224946,69484,1.000


In [31]:
score_min = 0.5  #Fixer un score minimum

reco_top_df = reco_top_df[reco_top_df['score_similarite'] >= score_min]


In [32]:
reco_top_df

,j_id,bv_id_recommande,score_similarite
0,190833,69827,0.583
1,190833,64427,0.562
2,190833,70968,0.554
6,191226,70144,0.701
9,191939,69484,1.000
...,...,...,...
14736,221071,69484,1.000
14739,223615,69484,1.000
14742,223618,69484,1.000
14745,224946,69484,1.000


In [33]:
def niveau(score): #Ajouter un niveau de compatibilité pour faciliter les filtrage
    if score == 1:
        return "Parfait"
    elif score >= 0.7:
        return "Très bon"
    elif score >= 0.4:
        return "Moyen"
    elif score > 0:
        return "Faible"
    else:
        return "Aucun match"


In [36]:
reco_df["niveau_compatibilite"] = reco_df["score_similarite"].apply(niveau)
reco_df

,j_id,bv_id_recommande,score_similarite,niveau_compatibilite
0,190833,69827,0.583,Moyen
2,191226,70144,0.701,Très bon
3,191939,69484,1.000,Parfait
6,195609,69484,1.000,Parfait
7,195782,69484,1.000,Parfait
...,...,...,...,...
4912,221071,69484,1.000,Parfait
4913,223615,69484,1.000,Parfait
4914,223618,69484,1.000,Parfait
4915,224946,69484,1.000,Parfait


#### Chargement des Données

In [39]:
# Nom de la table de destination
table_id = "article1-data-pipeline.article1_dbt_mart.mart_recommandations"

# Écrire dans BigQuery (écrasement complet de la table)
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # remplace toute la table
    autodetect=True,  # infère automatiquement le schéma
)

# Lancer le job d'import
job = client.load_table_from_dataframe(reco_df, table_id, job_config=job_config)
job.result()  # attendre la fin du job

print(f"Données chargées dans {table_id} ({reco_df.shape[0]} lignes).")


C:\Users\awadi\anaconda3\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Données chargées dans article1-data-pipeline.article1_dbt_mart.mart_recommandations (2868 lignes).
